In [ ]:
# %%
import sys
import os
import luigi
import sciluigi as sl
import logging

sys.path.append('..')
# everyone needs to be quite 
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('SMB').setLevel(logging.WARNING)
logging.getLogger('napari').setLevel(logging.WARNING)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('in_n_out').setLevel(logging.WARNING)
logging.getLogger('numcodecs').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('luigi').setLevel(logging.WARNING)
logging.getLogger('numexpr').setLevel(logging.WARNING)
logging.getLogger('luigi-interface').setLevel(logging.WARNING)
logging.getLogger('sciluigi-interface').setLevel(logging.WARNING)
logging.getLogger('cellpose').setLevel(logging.WARNING)



from src import Receipt
from src.steps import get_task

In [ ]:
from luigi.contrib.ssh import RemoteTarget

class AngelFISHLuigiTask(sl.ExternalTask):
    receipt_path = luigi.Parameter()
    step_name = luigi.Parameter()
    output_path = luigi.Parameter()

    @property
    def task_name(self):
        return self.step_name

    def out_doneflag(self):
        return RemoteTarget(self, self.output_path)

    def run(self):
        receipt = Receipt(path=self.receipt_path)
        task_class = get_task(receipt['steps'][self.step_name]['task_name'])
        task = task_class(receipt, self.step_name)

        command = f"sbatch run_step.sh {self.receipt_path} {self.step_name}"
        subprocess.run(['ssh', 'user@remote-hpc', command], check=True)

In [ ]:
class Upload_Task(sl.ExternalTask):
    pass # this needs to upload receipt_path to the cluster

In [ ]:
class AngelFISHWorkflow(sl.WorkflowTask):
    receipt_path = luigi.Parameter() # this location will get change along the execution of the workflow

    def workflow(self):
        # upload 






        receipt = Receipt(path=self.receipt_path)
        step_order = receipt['step_order']

        previous_task = None
        task_refs = []
        for step_name in step_order:
            path = os.path.join(receipt['dirs']['status_dir'], f'step_{step_name}.txt') # we need this to match the computer we are on
            step_task = self.new_task(
                step_name,
                AngelFISHLuigiTask,
                receipt_path=self.receipt_path,
                step_name=step_name,
                output_path=path
            )

            # Add dependency chain
            if previous_task is not None:
                step_task.in_upstream = previous_task.out_doneflag
            previous_task = step_task
            task_refs.append(step_task)
        return task_refs

In [4]:
luigi.build(AngelFISHWorkflow(receipt_path=r'C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json').workflow(), local_scheduler=True)

DEBUG: Checking if AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt) is complete
DEBUG:luigi-interface:Checking if AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt) is complete
2025-07-30 14:05:38 |     INFO | Informed scheduler that task   AngelFISHLuigiTask_download_data_c__Users_f

Connection established
File Downloaded : NDTiffStack.tif


2025-07-30 14:05:47 |     INFO | Task download_data finished after 9.196s
INFO:sciluigi-interface:Task download_data finished after 9.196s


File Downloaded : NDTiff.index
Files downloaded to: c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7


2025-07-30 14:05:48 |     INFO | [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt)
INFO: [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\da

...

2025-07-30 14:08:09 |     INFO | Task measure_cell_props finished after 2.012s
INFO:sciluigi-interface:Task measure_cell_props finished after 2.012s
2025-07-30 14:08:09 |     INFO | [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=measure_cell_props, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=measure_cell_props, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_measure_cell_props.txt)
INFO: [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name

Uploaded: cyto_masks.tiff
Uploaded: nuc_masks.tiff
Uploaded: cellproperties.csv
Uploaded: dusp1_cellresults.csv
Uploaded: dusp1_clusterresults.csv
Uploaded: dusp1_spotresults.csv
Uploaded: step_detect_dusp1.txt
Uploaded: step_download_data.txt


2025-07-30 14:08:16 |     INFO | Task return_data finished after 7.272s
INFO:sciluigi-interface:Task return_data finished after 7.272s
2025-07-30 14:08:16 |     INFO | [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=return_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=return_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_return_data.txt)
INFO: [pid 25180] Worker Worker(salt=2959556853, workers=1, host=ens13487, username=formanj, pid=25180) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=return_data, receipt_path=C:\Users

Uploaded: step_match_cyto_and_nuc.txt
Uploaded: step_measure_cell_props.txt
Uploaded: step_segment_cyto.txt
Uploaded: step_segment_nuc.txt


True